#### Handy tools and stuff I use all the time.
Hint: git is set up to ignore notebooks that begin with the name "Untitled".  So copy this notebook to "Untitled_Tools.ipynb" and git won't bug you about it.

In [3]:
# delete or comment out this cell if you aren't using autorun
# or, install autorun, and you won't have to scroll to the top of the notebook every time you restart :-)
from notebook_autorun import Autorun
Autorun(cells=None, metadata=False, comment=True, comment_flag='AUTORUN', verbose=True).add_js()

If you see <IPython.core.display.Javascript object> below, this notebook is not trusted.
As a consequence Autorun cannot work.
Run "from notebook_autorun import Autorun; Autorun.info()" for more info.


<IPython.core.display.Javascript object>

This output cell contains **notebook-autorun** settings:  
```json
   {"comment": true, "comment_flag": "AUTORUN"}  
```

In [10]:
# AUTORUN

# import sys
# sys.path.append('../')   # Add root to path if you need to.

import pandas as pd
import numpy as np
from pathlib import Path
import pytest
import importlib
import openpyxl
import json
import sys
from tools import expected_result_tester as ert
from tools import expected_ghost as ghost
from tools import solution_xls_extract as sxe
from tools import multi_excel_sample as mex
from tools import create_expected_zip as cez
from tools import vma_xls_extract as vxe
from tools import util
from solution import factory
import model.advanced_controls as ac
import model.dd as dd
import model.scenario as scenario
import model.customadoption as ca
import model.tam as tam

pd.set_option('display.float_format', lambda x: f"{x:03.5f}".rstrip('0'))

# Extraction
Steps to create a python model from an Excel model

In [7]:
# Set These
#excelname="SmartThermostats-RRSv1.1b-Oct2019.xlsm"
#excelname="SolarPVDistributed_RRS_ELECGEN_v1.1b_26Jun2020.xlsm"
excelname="RecycledMetals_RRS_v1_2021.xlsm"
solutionname="recycledmetals"

exceldir=Path("C:/Working/ModelsAll")
#exceldir=Path("G:/My Drive/AllModels")
#codedir=Path("C:/Working/solutions/solution")
codedir=Path(r"C:/Working/UpdateSample")

# don't change these
excelfile= exceldir/excelname
zipfile=exceldir/f"{solutionname}_expected.zip"
solutiondir=codedir/solutionname

In [7]:
# This may already have been done for you, but if not:

# Step 1:  Open the spreadsheet, add the export macro, and run it.  
# *** Save the spreadsheet (it may have been udpated!) ***
# This has to be done first, since it may modify the spreadsheet
# (more detailed instructions in the Extraction Guide)

# Step 2: Create the expected.zip

cez.create_expected_zip(exceldir)                              # Create the expected.zip file
(exceldir/"expected.zip").rename(zipfile)                      # Rename zipfile temporarily        
[f.unlink() for f in exceldir.glob("*.xlsm_*")]                # Clean up all the extra files
print(f"{solutionname} done: {zipfile.is_file()}")

solarpvroof done: True


In [8]:
# Step 3: Run the extractor

sxe.output_solution_python_file(outputdir=solutiondir,xl_filename=excelfile)

In [14]:
# Step 4
# If the .zip hasn't been copied over before, do it now

import shutil
testdir = solutiondir/"tests"
testdir.mkdir(exist_ok=True)
shutil.copy(zipfile, testdir/"expected.zip")

# Only copy the test template if there isn't one already, to preserve any customizations
testfile=testdir/f"test_{solutionname}.py"
if not testfile.is_file():
    shutil.copy(solutiondir.parents[1]/"tools/solution_test_template.py", testfile)

In [15]:
# Step 5
# Review the __init__.py file, or merge the old __init__.py and the __init__UPDATED.py files

# You must set the PDS1, PDS2, PDS3 vars
# These are your options:

factory.list_scenarios(solutionname)

# Also: Delete all the older scenarios: we really only care about having the most recent three.

['PDS-10p2050-Drawdown (Book Ed.1)',
 'PDS-10p2050-Optimum (Book Ed. 1)',
 'PDS-14p2050-Drawdown2020',
 'PDS-14p2050-Optimum2020',
 'PDS-14p2050-PDS1_June2020',
 'PDS-14p2050-PDS2_June2020',
 'PDS-14p2050-PDS3_June2020',
 'PDS-14p2050-Plausible2020',
 'PDS-7p2050-Plausible (Book Ed. 1)']

# Test

In [2]:
import solution.smartthermostats.tests.test_smartthermostats as st
st.test_deep_results()

Checking scenario 0: PDS1-28p2050-cs Low-E12.84, F8.89 (Book Ed.1)
Checking scenario 1: PDS2-39p2050-cs-Avg E11.65, F8.06 (Book Ed.1)
Checking scenario 2: PDS3-50p2050-cs-High E10.96, F7.59 (Book Ed.1)


In [2]:
# run the entire test suite
!pytest --tb=short > ..\testruns\trimtests.out

In [4]:
# I keep test runs so I can do this diff.  I also add the test output to releases, so you can grab an older
# test result to compare to.
# (Someday, when expected_result_tester isn't our workhorse, we should just switch to a modern test result tracker.)

!tools\diff_testruns.py ..\testruns\v23.out ..\testruns\justchecking.out

============================= test session starts =============================
platform win32 -- Python 3.9.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Date: 2021-10-14 12:29:04.868372
Git: v0.22.0-14-g94ed51f3 (vmafiles)
rootdir: C:\Working\solutions, configfile: tox.ini
collected 1355 items
============================= test session starts =============================
platform win32 -- Python 3.9.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Date: 2021-10-22 10:53:44.064362
Git: v0.23.0-16-g77347719 (scenario)
rootdir: C:\Working\solutions, configfile: tox.ini
collected 1354 items


GOT WORSE:

helpertables.py: ........................./..FF...s.FFFFF..FFF...F..
commercialglass.py: .../.FF
composting.py: .../..F
recycledplastics.py: .../..F
smartthermostats.py: .../.FF

DIFFERENT:

advanced_controls.py: ..............F....F......... / ...........................
s_curve.py: ...... / .......
residentialglass.py: FFF / .FF


In [21]:
# another way to summarize the test data.

!python -m tools.summarize_expected_result -v -m solarpvroof ../testruns/justchecking.out

solarpvroof key
    0: marginal_first_cost,net_operating_savings,lifetime_operating_savings
    1: marginal_first_cost,net_operating_savings,lifetime_operating_savings
    2: marginal_first_cost,net_operating_savings,lifetime_operating_savings
    3: marginal_first_cost,net_operating_savings,lifetime_operating_savings
solarpvroof deep
    0:Helper Tables B27:L73;Unit Adoption Calculations AG199:AQ244,B252:L298,Q252:AA298,Q198:AA244,AX198:BH244,AG253:AQ298,AX252:BH298;First Cost L37:L82,N37:N82
    1:Helper Tables B27:L73;Unit Adoption Calculations AG199:AQ244,B252:L298,Q252:AA298,Q198:AA244,AX198:BH244,AG253:AQ298,AX252:BH298;First Cost L37:L82,N37:N82
    2:Helper Tables B27:L73;Unit Adoption Calculations AG199:AQ244,B252:L298,Q252:AA298,Q198:AA244,AX198:BH244,AG253:AQ298,AX252:BH298;First Cost L37:L82,N37:N82
    3:Helper Tables B91:L137,B27:L73;Unit Adoption Calculations AG199:AQ244,B252:L298,Q252:AA298,Q135:AA181,AX136:BH182,AG137:AQ182,Q198:AA244,AX198:BH244


In [ ]:
foo = factory.load_scenario("smartthermostats","PDS1-28p2050-cs Low-E12.84, F8.89 (Book Ed.1)")

# Multi Extract
Makes a single notebook extracing the same section from a tab in lots of notebooks.  Good for comparing how different models do things.  With `copy_data=True`: show the data in the region.  With `copy_formla=True`, show the formulas in the region.

In [ ]:
newmodels = list(Path(r"..\ModelsNew").glob("*.xlsm"))
existingmodels = list(Path(r"..\ModelsExisting").glob("*.xlsm"))
anintegration = list(Path(r"..\Integrations").glob("ELC*.xlsx"))
allmodels = list(Path(r"..\ModelsAll").glob("*.xlsm"))

nobio = sorted([ x for x in newmodels+existingmodels if 'BIOS' not in x.name ])

In [ ]:
page = "Helper Tables"
region = "B20:B22"

wb = mex.sample_regions(allmodels, page, region, copy_formula=False)

DATA
AirplaneFE-RRSv1.1c-June2020.xlsm
AlternativeCements_v1.1c_IntegrationJune2020.xlsm
BikeInfra-RRSv1.1c-7Oct2019.xlsm
BiomassPower_RRS_ELECGEN_26June2020.xlsm
Bioplastics-RRS-MRGJune2020Integration.xlsm
BuildingAutomation_v1.1c_Nov2019.xlsm
Carpool-RRSv1.1b-Jun2020.xlsm
CleanCookstoves-PDRRSv.1.1b-24Oct2019.xlsm
Composting-v1.1b-MRG2020Integration.xlsm
CSP_RRS_ELECGEN_v1.1b_26Jun2020.xlsm
District Heating_RRS_v1.1c_Oct2019.xlsm
Drawdown_RRS-BIOSEQAgri_Model_v1.1b_Conservation_Agriculture_Aug2020.xlsm
Drawdown_RRS-BIOSEQAgri_Model_v1.1b_TreeIntercropping_Jun2020.xlsm
Drawdown_RRS-BIOSEQAgri_Model_v1.1_Improved Rice_Jun2020.xlsm
Drawdown_RRS-BIOSEQAgri_Model_v1.1_Regenerative Annual Cropping_Jun2020.xlsm
Drawdown_RRS-BIOSEQAgri_Model_v1.1_SRI_Jun2020.xlsm
Drawdown_RRS-BIOSEQAgri_Model_v1.1_Sustainable Intensification for Smallholders_Jan2020.xlsm
Drawdown_RRS-BIOSEQProtect_Model_v1.1b_Coastal_Wetlands(Mangroves)_Jun2020.xlsm
Drawdown_RRS-BIOSEQProtect_Model_v1.1b_Coastal_Wetlands(Sal

In [ ]:
wb.save(r'..\Samples\ht_baseyear.xlsx')
wb.close()

# Ghost Maker
Make a spreadsheet from an expected.zip file (data only, no formulas).  Useful to see exactly what the tests see.

In [ ]:
zf = ghost.locate_expected_zip('electricvehicles')
gh = ghost.create_ghost(zf, 0)    # zero-based scenario number

In [ ]:
gh.save(r'..\Ghosts\evs0.xlsx')
gh.close()

# Programatic Solution Editing
If you have to edit all the solution files, it can be lots easier to do it this way...

In [3]:
def count_indent(str):
    return len(str) - len(str.lstrip())

def next_empty_line(lines, i):
    for j in range(i+1,len(lines)):
        if lines[j].strip() == "":
            return j
    raise ValueError("not found")
        
def next_line_at_indent(lines, i):
    indent = count_indent(lines[i])
    for j in range(i+1,len(lines)):
        if count_indent(lines[j]) == indent:
            return j
    raise ValueError("not found")

def line_containing(lines, val, start=0):
    for i in range(start, len(lines)):
        if val in lines[i]:
            return i
    raise ValueError("not found")

def match_indent(otherline, newcontent):
    indent = count_indent(otherline)
    newlines = [ " "*indent + x for x in newcontent.splitlines() ]
    return "\n".join(newlines)

def insert_lines(lines, i, newlines):
    return lines[:i] + newlines.splitlines() + lines[i:]

def delete_lines(lines, i, j):
    return lines[:i] + lines[j:]

def comment_around_block(lines, i, j):
    cmt1 = [match_indent(lines[i], "# #BEGIN COMMENT BLOCK")]
    cmt2 = [match_indent(lines[i], "# #END COMMENT BLOCK")]
    return lines[:i] + cmt1  + lines[i:j] + cmt2 + lines[j:]

def comment_out_block(lines, i, j):
    lines = lines[:i] + [ match_indent(lines[i], '#' + txt.lstrip()) for txt in lines[i:j] ] + lines[j:]
    return comment_around_block(lines, i, j)


In [12]:

def update_tam_1(sf):
    filetext = sf.read_text(encoding='utf-8')
    if "_ref_tam_sources" in filetext:
        lines = filetext.splitlines()
        refline = line_containing(lines, "_ref_tam_sources")
        if "rrs" in lines[refline]:
            lines[refline] = match_indent(lines[refline], 
                    "_ref_tam_sources = scenario.load_and_root(DATADIR/'energy'/'ref_tam_2_sources.json','*')")
            lines[refline+1] = match_indent(lines[refline], 
                    "_pds_tam_sources = scenario.load_and_root(DATADIR/'energy'/'pds_tam_2_sources.json','*')")
            sf.write_text("\n".join(lines), encoding='utf-8')

def update_adoption(sf):
    filetext = sf.read_text(encoding='utf-8')
    lines = filetext.splitlines()
    sourceline = line_containing(lines, "__init__")-1
    # for these we check by looking for the directory, rather than in the code
    if (sf.parent/'ad').is_dir():
        newtext = "    _pds_ad_sources = scenario.load_sources(THISDIR/'ad'/'ad_sources.json', '*')"
        lines = insert_lines(lines, sourceline, newtext)
        start = line_containing(lines, "adconfig_list")    
        end = line_containing(lines, "adconfig=adconfig") + 1
        lines = comment_around_block(lines, start, end)
    
    if (sf.parent/'ca_pds_data').is_dir():
        newtext = "    _pds_ca_sources = scenario.load_sources(THISDIR/'ca_pds_data'/'ca_pds_sources.json', 'filename')"
        lines = insert_lines(lines, sourceline, newtext)
        start = line_containing(lines,"ca_pds_data_sources")
        end = next_empty_line(lines, start)
        lines = comment_around_block(lines, start,end)
    
    if (sf.parent/'ca_ref_data').is_dir():
        newtext = "    _ref_ca_sources = scenario.load_sources(THISDIR/'ca_ref_data'/'ca_ref_sources.json', 'filename')"
        lines = insert_lines(lines, sourceline, newtext)
        start = line_containing(lines," # Custom REF Data")
        end = next_empty_line(lines, start)
        lines = comment_around_block(lines, start,end)

    outputline = line_containing(lines,"if False")-2
    lines = insert_lines(lines, outputline, "        self.initialize_adoption_bases()")
    sf.write_text("\n".join(lines), encoding='utf-8')

def move_jsons_into_init(sf):
    filetext = sf.read_text(encoding='utf-8')
    lines = filetext.splitlines()
    try:
        startline = line_containing(lines, "module_name = THISDIR.stem")+1
        endline = line_containing(lines, "__init__(")-1
        movelines = lines[startline:endline]
        del( lines[startline:endline] )

        insertpoint = line_containing(lines, "__init__(")+1
        indent = count_indent(lines[insertpoint-1]) + 4
        # strip out empty lines
        movelines = [ line for line in movelines if len(line.strip()) ]
        movelines = [""] + [ " "*indent + "self." + line.lstrip() for line in movelines ] + [""]
        lines[insertpoint:insertpoint] = movelines
        sf.write_text("\n".join(lines), encoding='utf-8')
    except:
        print(f"skipping {sf}")

def replace_vma_struct(sf):
    filetext = sf.read_text(encoding='utf-8')
    lines = filetext.splitlines()
    startline = line_containing(lines, "VMAs =")
    endline = line_containing(lines, "vma.populate_fixed_summaries")
    lines[startline:endline+1] = ["VMAs = vma.VMA.load_vma_directory(THISDIR/'vma_data/vma_sources.json')"]
    sf.write_text("\n".join(lines), encoding='utf-8')

In [13]:
solution_dir = Path("solution")
def edit_all():
    solution_init_files = solution_dir.glob('*/__init__.py')
    for sf in solution_init_files:
        replace_vma_struct(sf)


In [14]:
edit_all()

In [9]:
def delete_pds_base_adoption(sf):
    filetext = sf.read_text(encoding='utf-8')
    if "pds_base_adoption" not in filetext:
        return
    lines = filetext.splitlines()
    startline = line_containing(lines, "pds_base_adoption")
    endline = line_containing(lines, "soln_pds_adoption_basis")
    del( lines[startline:endline] )
    sf.write_text("\n".join(lines), encoding='utf-8')

In [10]:
# Try it out once
delete_pds_base_adoption( Path(r'solution\coolroofs\ac\PDS1-61p2050-Integrated-EE872%-FE-281%_Book.json'))

In [11]:
# Then for real
for sf in Path("solution").glob("*/ac/*.json"):
    delete_pds_base_adoption(sf)


# Open Season
Marks the end of the prepared contents.

In [2]:
foo = factory.load_scenario("smartthermostats","PDS3")

In [5]:
for f in Path("solution").glob("*/vma_data/VMA_info.csv"):
    f.unlink()